In [74]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [159]:
# Lê a primeira coluna do CSV de resultados para ver instâncias já resolvidas
try:
    instancias_resolvidas = [row[0] for row in csv.reader(open('results.csv'))][1:]  # Exclui o cabeçalho
except IndexError:
    instancias_resolvidas = []

folder_path = 'instancias/maria_desiree/'
instancias = natsorted([f for f in os.listdir(folder_path) if f.startswith('AAA01226') and f.replace('.dat', '') not in instancias_resolvidas])  # Instâncias a serem resolvidas

print(instancias)

['AAA01226_2.dat']


# Leitura de instâncias

In [75]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1
    
    for i in range(periods):  # Lê a linha de demandas para o período
        demands = list(map(int, lines[demand_start_line + i].split()))
        period_demand = []  # Divide as demandas entre as plantas
        for j in range(num_plants):
            # Extraindo demandas por planta (assumindo que cada planta tem um número igual de itens)
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)

        demand_matrix.append(period_demand)
    demand_matrix = np.array(demand_matrix)
    demand_matrix = np.transpose(demand_matrix, (2, 1, 0))

    # 7. Lendo os custos de transferência
    transfer_costs = []
    transfer_cost_line = demand_start_line + periods
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()  # Verificar se a linha não está vazia antes de tentar ler
        if line:
            transfer_costs.append(float(line))
        transfer_cost_line += 1

    def create_transfer_cost_matrix(transfer_costs, num_plants):  # Criar a matriz de custos de transferência (simétrica)
        transfer_cost_matrix = np.zeros((num_plants, num_plants))  # Inicializar a matriz de zeros
        if len(transfer_costs) == 1:
            transfer_cost = transfer_costs[0]  # Se houver apenas um custo de transferência, aplicar para todos os pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_cost
                    transfer_cost_matrix[k, j] = transfer_cost
        else:
            idx = 0  # Se houver múltiplos custos, aplicar entre pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_costs[idx]
                    transfer_cost_matrix[k, j] = transfer_costs[idx]
                    idx += 1
        return transfer_cost_matrix

    transfer_costs = create_transfer_cost_matrix(transfer_costs, num_plants)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": demand_matrix,
            "transfer_costs": transfer_costs}

In [76]:
# Exemplo de uso
file_path = 'instancias/maria_desiree/AAA01226_0.dat'
data = read_dat_file(file_path)
# display(data)

# Modelagem

In [77]:
m = gp.Model('Lot-sizing Sambasivan and Yahya')

## Conjuntos

In [78]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

## Parâmetros

In [79]:
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

## Variáveis de decisão

In [80]:
# Quantidade produzida (i, j, t)
X = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='X')
# Quantidade estocada (i, j, t)
Q = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='Q')
# # Quantidade transportada (i, j, k(um outro j), t)
W = m.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# # Variável de setup (binária) (i, j, t)
Z = m.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

## Função objetivo

In [81]:
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                            sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
m.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

## Restrições

In [82]:
# Balanço de estoque (revisar comportamento)
# Período inicial
m.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
             name='restricao_balanco_estoque')
# Demais períodos
m.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
             name='restricao_balanco_estoque');

In [83]:
# Restrição que obriga setup (validar o range do r)
m.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
             , name='restricao_setup');

In [84]:
# Restrição de capacidade
m.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

# Resolução

In [85]:
# Critérios de parada
m.Params.timelimit = 3600  # Tempo (default = inf)
# m.Params.MIPgap = 0.01  # Gap de otimalidade (default = 0.0001 = 0.01%)

# Otimização
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 312 rows, 720 columns and 1284 nonzeros
Model fingerprint: 0x70ecf60e
Variable types: 576 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e-01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+03]
Presolve removed 0 rows and 156 columns
Presolve time: 0.00s
Presolved: 312 rows, 564 columns, 1272 nonzeros
Variable types: 420 continuous, 144 integer (144 binary)

Root relaxation: objective 3.855114e+04, 546 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 38551.1395    0   

In [97]:
# Nome do teste
m.ModelName

'Lot-sizing Sambasivan and Yahya'

In [99]:
# Número de períodos da instância
len(T)

12

In [100]:
# Número de fábricas
len(J)

2

In [101]:
# Número de produtos
len(I)

6

In [86]:
# Tempo de execução em segundos
m.Runtime

114.98000001907349

In [87]:
# Valor da função objetivo da melhor solução encontrada (upper bound da minimização)
m.ObjVal

46248.14733082707

In [88]:
# Lower bound da minimização
m.ObjBound

46248.14733082707

In [89]:
# Gap da solução
m.MIPgap

0.0

In [90]:
m.Status  # Ver https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes para interpretar códigos

2